In [9]:
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

env = gym.make("CartPole-v1")

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)

In [10]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [11]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [12]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

# Get number of actions from gym action space
# action_space = env.action_space
# n_actions = action_space.n

# Define action space
action_space = gym.spaces.Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)
n_actions = np.prod(action_space.shape)
# Get the number of state observations
# state, info = env.reset()
# n_observations = len(state)
n_observations = 5

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(100000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            # return policy_net(state).max(1).indices.view(1, 1)
            action = policy_net(state).view(1, 1, 1)
            return action
    else:
        return torch.tensor([[action_space.sample()]], device=device, dtype=torch.float32)


episode_durations = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [13]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    print(transitions[-1])
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [14]:
from controllers import BaseController

last_action = 0


class Controller(BaseController):
    def update(self, target_lataccel, current_lataccel, state, future_plan):
        global last_action
        inputs = (target_lataccel, current_lataccel, *state)
        inputs = torch.tensor(inputs, device=device, dtype=torch.float32)
        action = select_action(inputs)
        last_action = action
        return float(action.flatten()[0].cpu())

In [15]:
from pathlib import Path, PurePosixPath
import tinyphysics as uphy
from hashlib import md5

MODEL_PATH = "./models/tinyphysics.onnx"
DATA_PATH = "./data"
sim: uphy.TinyPhysicsSimulator = None


def reset_sim(seg: int, model_path=MODEL_PATH, data_dir=DATA_PATH, debug=False, seed=None):
    global sim
    model = uphy.TinyPhysicsModel(model_path, debug=debug)
    data_dir = Path(data_dir)
    controller = Controller()
    file = data_dir / f"{seg:05d}.csv"
    # normalize path to the posix path
    if not file.is_absolute():
        file = PurePosixPath(file)
    sim = uphy.TinyPhysicsSimulator(model, str(file), controller=controller, debug=debug)
    if seed is not None:
        seed = int(md5(str(seed).encode()).hexdigest(), 16) % 10**4
        uphy.np.random.seed(seed)

    for _ in range(uphy.CONTEXT_LENGTH, uphy.CONTROL_START_IDX - 1):
        sim.step()

    state = sim.target_lataccel_history[-1], sim.current_lataccel, *sim.state_history[-1]
    return state, None


def sim_step():
    sim.step()
    state = sim.target_lataccel_history[-1], sim.current_lataccel, *sim.state_history[-1]
    reward = sim.target_lataccel_history[-1] - sim.current_lataccel
    truncated = sim.step_idx >= len(sim.data) - uphy.CONTEXT_LENGTH
    return state, reward, False, truncated, None

In [16]:
if torch.cuda.is_available() or torch.backends.mps.is_available():
    num_episodes = 600
else:
    num_episodes = 50

segments = random.sample(range(num_episodes), num_episodes)

for _episode_i, segment_no in enumerate(segments):
    # Initialize the environment and get its state
    state, _info = reset_sim(segment_no)
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    print("past the reset\n\n")
    for t in count():
        print(f"t: {t}")
        # action = select_action(state)
        # observation, reward, terminated, truncated, _ = env.step(action.item())
        observation, reward, terminated, truncated, _ = sim_step()
        action = last_action
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

past the reset


t: 0
t: 1
t: 2
t: 3
t: 4
t: 5
t: 6
t: 7
t: 8
t: 9
t: 10
t: 11
t: 12
t: 13
t: 14
t: 15
t: 16
t: 17
t: 18
t: 19
t: 20
t: 21
t: 22
t: 23
t: 24
t: 25
t: 26
t: 27
t: 28
t: 29
t: 30
t: 31
t: 32
t: 33
t: 34
t: 35
t: 36
t: 37
t: 38
t: 39
t: 40
t: 41
t: 42
t: 43
t: 44
t: 45
t: 46
t: 47
t: 48
t: 49
t: 50
t: 51
t: 52
t: 53
t: 54
t: 55
t: 56
t: 57
t: 58
t: 59
t: 60
t: 61
t: 62
t: 63
t: 64
t: 65
t: 66
t: 67
t: 68
t: 69
t: 70
t: 71
t: 72
t: 73
t: 74
t: 75
t: 76
t: 77
t: 78
t: 79
t: 80
t: 81
t: 82
t: 83
t: 84
t: 85
t: 86
t: 87
t: 88
t: 89
t: 90
t: 91
t: 92
t: 93
t: 94
t: 95
t: 96
t: 97
t: 98
t: 99
t: 100
t: 101
t: 102
t: 103
t: 104
t: 105
t: 106
t: 107
t: 108
t: 109
t: 110
t: 111
t: 112
t: 113
t: 114
t: 115
t: 116
t: 117
t: 118
t: 119
t: 120
t: 121
t: 122
t: 123
t: 124
t: 125
t: 126
t: 127
Transition(state=tensor([[ 0.8863,  0.8553, -0.0713, 12.3325, -0.3216]], device='cuda:0'), action=tensor([[[-0.7986]]], device='cuda:0'), next_state=tensor([[ 0.8944,  0.8749, -0.0737, 12.3112, -0.

RuntimeError: gather(): Expected dtype int64 for index